In [9]:
from google.colab import drive
drive.mount('/content/drive')#, force_remount=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
%%capture
!pip install GOES
!pip install pyresample

In [11]:
import datetime
import os

import numpy as np
import GOES
import pyproj as pyproj
from pyresample import utils
from pyresample.geometry import SwathDefinition
from pyresample.kd_tree import resample_nearest
from pyresample import bilinear
from netCDF4 import Dataset, date2num

In [12]:
def get_bucket(Satellite, Product, DateTime, channel):
    DateTime = datetime.datetime.strptime(DateTime,'%Y%m%d-%H%M%S')
    pattern = 'gs://gcp-public-data-goes-'+Satellite+'/'+Product+DateTime.strftime('/%Y/%j/%H/')+'*C'+channel+'*s'+DateTime.strftime('%Y%j%H')+'*'
    list = !gsutil ls $pattern
    return list;

In [13]:
def reproject(CMI, LonCen, LatCen, LonCenCyl, LatCenCyl):
    Prj = pyproj.Proj('+proj=eqc +lat_ts=0 +lat_0=0 +lon_0=0 +x_0=0 +y_0=0 +a=6378.137 +b=6378.137 +units=km')
    AreaID = 'cyl'
    AreaName = 'cyl'
    ProjID = 'cyl'
    Proj4Args = '+proj=eqc +lat_ts=0 +lat_0=0 +lon_0=0 +x_0=0 +y_0=0 +a=6378.137 +b=6378.137 +units=km'

    ny, nx = LonCenCyl.shape
    SW = Prj(LonCenCyl.min(), LatCenCyl.min())
    NE = Prj(LonCenCyl.max(), LatCenCyl.max())
    area_extent = [SW[0], SW[1], NE[0], NE[1]]

    AreaDef = utils.get_area_def(AreaID, AreaName, ProjID, Proj4Args, nx, ny, area_extent)

    SwathDef = SwathDefinition(lons=LonCen, lats=LatCen)
    #CMICyl = resample_nearest(SwathDef, CMI, AreaDef, radius_of_influence=6000, fill_value=np.nan, epsilon=0, reduce_data=False)
    CMICyl = bilinear.resample_bilinear(CMI, SwathDef, AreaDef, radius=6000, fill_value=np.nan, reduce_data=False, segments=None, epsilon=0) # nprocs=1, neighbours=32

    return CMICyl;

In [14]:
def save_density_as_nc(Field, FieldName, LonsCen, LatsCen, DateTimeField, NameOutputFile, OutputPath=''):

   # creates netcdf file
   dataset = Dataset(OutputPath+NameOutputFile, 'w', format='NETCDF4')

   # Dimensions - Is necessary that dimension have this names
   dataset.createDimension('time', None)
   #dataset.createDimension('level', None)
   dataset.createDimension('latitude', LatsCen.shape[0])
   dataset.createDimension('longitude', LonsCen.shape[1])

   # Variables - Is necessary that dimension variables have this names
   times_file = dataset.createVariable('time', np.float64, ('time',))
   #level_file = dataset.createVariable('level', np.float32, ('level',))
   lats_file = dataset.createVariable('latitude', LatsCen.dtype.type, ('latitude',))
   lons_file = dataset.createVariable('longitude', LonsCen.dtype.type, ('longitude',))

   parameter01 = dataset.createVariable('CMI', Field.dtype.type, ('latitude', 'longitude'), zlib=True) #, least_significant_digit=2)

   # Variable attributes
   times_file.standard_name = 'time'
   times_file.long_name = 'time'
   times_file.units = 'hours since 2000-1-1 00:00:00'
   times_file.calendar = 'standard'
   times_file.axis = 'T'

   #level_file.standard_name = 'level'
   #level_file.long_name = 'level'
   #level_file.units = 'millibars'
   #level_file.positive = 'down'
   #level_file.axis = 'Z'

   lats_file.standard_name = 'latitude'
   lats_file.long_name = 'latitude'
   lats_file.units = 'degrees_north'
   lats_file.axis = 'Y'

   lons_file.standard_name = 'longitude'
   lons_file.long_name = 'longitude'
   lons_file.units = 'degrees_east'
   lons_file.axis = 'X'

   parameter01.standard_name = 'BT in CYl'#'{} density'.format(FieldName)
   parameter01.long_name = 'BT' #'{} density of GLM'.format(FieldName)
   parameter01.units = 'K' #'{} CMI in 2x2 Km'.format(FieldName)
   parameter01.axis = 'YX'

   # Writing variables
   times_file[:] = date2num(DateTimeField, units = times_file.units, calendar = times_file.calendar)
   #level_file[:] = 1000.0 #level0[:]
   lats_file[:] = LatsCen[:,0]
   lons_file[:] = LonsCen[0,:]
   parameter01[:,:] = Field

   # Global attributes
   dataset.description = 'GOES-16 Satellite'
   dataset.source = 'NOAA-NASA'
   dataset.author = 'Joao Henry Huaman Chinchay (joaohenry23@gmail.com)'
   dataset.close()

In [15]:
path_download = ''
path_output = '/content/drive/MyDrive/GOES16/ABI_Cyl/'

'''
CH02pixresol = 1.0 # resolution of fileout
'''

# this parameters are used to create a GLM navigation of 2D, then navigation is changed to 1 km
pixresol = 2.0
domain = [-88.0,-63.0,-25.0,5.0] #[-90.0,-30.0,-60.0,15.0]
xmin, xmax = 80, 1030 # -83.54954954954955 -66.45045045045045
ymin, ymax = 700, 1900 # -20.225225225225227 1.3783783783783794

ch = '13' # valid channels: '07','08','09','10','11',...,'16'
MinFileSize = 10.0 # Megabytes
DateTimeIni = '20180101-000000'
DateTimeFin = '20180101-010000'


time = datetime.datetime.now()

# ------------------
# here define the corners of pixels where will accumulate the lightnings

lat_cor = 14.0+np.arange(3665)*(-pixresol/111.0) #np.arange(14.0,-52.01,-pixresol/111.0)
lon_cor = -85.0+np.arange(2945)*(pixresol/111.0) #-84.0+np.arange(2897)*(pixresol/111.0) #-84.0+np.arange(2889)*(pixresol/111.0) #np.arange(-84.0,-31.98,pixresol/111.0)
lat_cen = lat_cor[:-1]-(pixresol/2.0)/111.0
lon_cen = lon_cor[:-1]+(pixresol/2.0)/111.0
lon_cor, lat_cor = np.meshgrid(lon_cor,lat_cor)
lon_cen, lat_cen = np.meshgrid(lon_cen,lat_cen)

lon_cor, lat_cor = lon_cor[ymin:ymax+1,xmin:xmax+1], lat_cor[ymin:ymax+1,xmin:xmax+1]
lon_cen, lat_cen = lon_cen[ymin:ymax,xmin:xmax], lat_cen[ymin:ymax,xmin:xmax]

'''
npy, npx = lon_cen.shape
lat_cor1km_all = np.linspace(lat_cor.max(),lat_cor.min(),int(npy*4/CH02pixresol+1))
lon_cor1km_all = np.linspace(lon_cor.min(),lon_cor.max(),int(npx*4/CH02pixresol+1))
lon_cen, lat_cen = np.meshgrid(lon_cor1km_all[1::2], lat_cor1km_all[1::2])
lon_cor, lat_cor = np.meshgrid(lon_cor1km_all[0::2], lat_cor1km_all[0::2])


del lat_cor1km_all, lon_cor1km_all
'''

#print('Cyl domain:')
#print(lon_cen.min(),lon_cen.max())
#print(lat_cen.min(),lat_cen.max())
#print(' ')
# ------------------

DateTimeIni = datetime.datetime.strptime(DateTimeIni,'%Y%m%d-%H%M%S')
DateTimeFin = datetime.datetime.strptime(DateTimeFin,'%Y%m%d-%H%M%S')

SatLon = None

while DateTimeIni <= DateTimeFin:

    print(' ')

    #if DateTimeIni.hour >= 10:

    print('{:%Y%m%d-%H%M}   -   {:%Y%m%d-%H%M}  -  \033[92m{}\033[0m'.format(DateTimeIni, DateTimeIni + datetime.timedelta(hours=1), 'Diurnal time'))
    files = get_bucket('16', 'ABI-L2-CMIPF', DateTimeIni.strftime('%Y%m%d-%H%M%S'), ch)

    if 'CommandException' in files[0]:
        print('  do not exist files')
    else:

        if os.path.exists(path_output+'C'+ch+DateTimeIni.strftime('/%Y/%m/'))==False:
            os.makedirs(path_output+'C'+ch+DateTimeIni.strftime('/%Y/%m/'))

        for file in files:

            !gsutil -q cp $path_download$file .

            FileSize = os.path.getsize(path_download+os.path.basename(file))*0.000001 # convert bytes to megabytes

            if FileSize > MinFileSize:

                ds = GOES.open_dataset(path_download+os.path.basename(file))

                SatLonFile = np.float32(ds.variable('nominal_satellite_subpoint_lon').data)
                if SatLon != SatLonFile:
                    SatLon = SatLonFile
                    print('  Calculates navigation!!!')
                    CMI, LonCen, LatCen = ds.image('CMI', lonlat='center', domain=domain)
                    domain_in_pixels = CMI.pixels_limits
                    mask = np.where(np.isnan(CMI.data)==True, True, False)

                    print(LonCen.data.shape)
                    '''
                    nx, ny = 2, 2
                    ysize, xsize = LonCen.data.shape
                    LonCen.data = np.mean(LonCen.data.reshape((int(ysize/ny),ny,int(xsize/nx),nx)), axis=(1,3))
                    LatCen.data = np.mean(LatCen.data.reshape((int(ysize/ny),ny,int(xsize/nx),nx)), axis=(1,3))
                    '''
                    print(LonCen.data.shape)

                else:
                    CMI, _, _ = ds.image('CMI', lonlat='none', domain_in_pixels=domain_in_pixels, nan_mask=mask)


                if CMI.time_bounds.data[0] >= DateTimeIni:

                    '''
                    CMI.data = np.mean(CMI.data.reshape((int(ysize/ny),ny,int(xsize/nx),nx)), axis=(1,3))
                    '''

                    print('  {}  {:6.2f} MB  {:%Y%m%d-%H%M}  \033[92m{}\033[0m'.format(os.path.basename(file),FileSize,CMI.time_bounds.data[0],'file accepted'))
                    CMICyl = reproject(CMI.data, LonCen.data, LatCen.data, lon_cen, lat_cen)
                    save_density_as_nc((CMICyl*1000).astype(np.int16), 'CMICyl', lon_cen, lat_cen, CMI.time_bounds.data[0], 'G16_C{}_Cyl_{:%Y%m%d-%H%M}.nc'.format(ch,CMI.time_bounds.data[0]), OutputPath=path_output+'C'+ch+(CMI.time_bounds.data[0]).strftime('/%Y/%m/'))
                    os.remove(os.path.basename(file))
                else:
                    print('  {}  \033[91m{:6.2f} MB\033[0m  \033[91m{:%Y%m%d-%H%M}\033[0m  \033[91m{}\033[0m'.format(os.path.basename(file),FileSize,CMI.time_bounds.data[0],'file rejected'))

            else:
                print('  {}  \033[91m{:6.2f} MB\033[0m  {}  \033[91m{}\033[0m'.format(os.path.basename(file),FileSize,'             ','file rejected'))


    #else:

    #    print('{:%Y%m%d-%H%M}   -   {:%Y%m%d-%H%M}  -  \033[91m{}\033[0m'.format(DateTimeIni, DateTimeIni + datetime.timedelta(hours=1), 'Nightly time'))


    DateTimeIni = DateTimeIni + datetime.timedelta(hours=1)

print(' ')
print((datetime.datetime.now() - time).seconds)


 
20180101-0000   -   20180101-0100  -  Diurnal time
  Calculates navigation!!!
(1588, 1371)
(1588, 1371)
  OR_ABI-L2-CMIPF-M3C13_G16_s20180010000387_e20180010011165_c20180010011246.nc   27.27 MB  20180101-0000  file accepted
  OR_ABI-L2-CMIPF-M3C13_G16_s20180010015387_e20180010026165_c20180010026246.nc   27.28 MB  20180101-0015  file accepted
  OR_ABI-L2-CMIPF-M3C13_G16_s20180010030387_e20180010041165_c20180010041245.nc   27.28 MB  20180101-0030  file accepted
  OR_ABI-L2-CMIPF-M3C13_G16_s20180010045387_e20180010056165_c20180010056242.nc   27.29 MB  20180101-0045  file accepted
 
20180101-0100   -   20180101-0200  -  Diurnal time
  OR_ABI-L2-CMIPF-M3C13_G16_s20180010100387_e20180010111165_c20180010111244.nc   27.29 MB  20180101-0100  file accepted
  OR_ABI-L2-CMIPF-M3C13_G16_s20180010115387_e20180010126166_c20180010126242.nc   27.30 MB  20180101-0115  file accepted
  OR_ABI-L2-CMIPF-M3C13_G16_s20180010130387_e20180010141165_c20180010141240.nc   27.31 MB  20180101-0130  file accepted
 